## Support Vector Machines (SVM)

Conjunto de modelos que se puede usar para:

   * clasificación
   
   * regresión
   
Algunas ventajas:

   * efectivas cuando hay un número de variables elevado
   
   * incluso si el número de variables es mucho mayor que el número de ejemplos
   
   * usa únicamente un subcojunto de las muestras de entrenamiento para predecir
   
   * versátiles: se pueden extender al caso no lineal via kernels
   
Algunas desventajas:

   * Rendimiento ligado al valor de los hiper-parámetros
   
   * No porducen estimaciones de la probabilidad de pertenencia a cada clase de forma directa
   
Las SVM lineales resuelven el problema de optimización:

$$\min_{w}\; C \sum_{i=1}^{n} \max\big\{0,\, 1 - y_i(w_0 + w_1x_1 + \dots + w_dx_d)\big\} + ||w||_2^2$$

Si lo comparamos con el problema de optimización de la **regresión logística regularizada**:

$$\min_{w}\; C\sum_{i=1}^{n} \log\big(\exp(-y_i(w_0 + w_1x_1 + \dots + w_dx_d)) + 1 \big) + ||w||_2^2$$

Son modelos muy similares, solo cambia la función de pérdida!

<img src=../../img/loss.png width=500>

[Guía de usuario](https://scikit-learn.org/stable/modules/svm.html)

### SVM lineal

scikit-learn implementa las SVM lineales en las clases:

 * `LinearSVC` para clasificación. Parámetros:
 
   1. `C`, cantidad de regularización
   
   2. `penalty`, si la regularización es $l_1$ o $l_2$
   
   
 * `LinearSVR` para regresión. Parámetros:
 
   1. `C`, cantidad de regularización 
   
   2. `epsilon`, parámetro de la función de pérdida
 
 
<img src=../../img/linear_svm_regularization.png width=500>

In [1]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_breast_cancer

X, y = load_breast_cancer(return_X_y=True)

svc = LinearSVC(C=1, max_iter=2000)
svc.fit(X, y)
svc.score(X, y)

/home/alberto/miniconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.9261862917398945

In [2]:
from sklearn.preprocessing import scale

X_std = scale(X)

svc = LinearSVC(C=1, max_iter=2000)
svc.fit(X_std, y)
svc.score(X_std, y)

0.9876977152899824

Las SVMs son sensibles al escalado de los datos, por lo que **siempre** es recomendable estandarizar.

En general podemos estandarizar a media 0 y desviación 1, pero si los datos tiene muchos 0 es mejor al intervalo $[-1, 1]$

### SVM no lineal

Crean fronteras de decisión no lineales transformando las variables originales a partir de una función de kernel:

<img src=../../img/nonlinear_svm.jpg width=500>

Implementación en scikit-learn:

  * `SVC` para clasificación. Parámetros:
  
    1. `C`, cantidad de regularización
    
    2. `kernel`, tipo de kernel. El más común es el kernel `rbf`
    
    3. Parámetros del kernel (`degree`, `gamma`, `coef0`)
  
  * `SVR` para regresión. Mismos parámetros que `SVC` y además:
  
    1. `epsilon`, parámetro de la función de pérdida
    
Recomendación para ajustar parámetros:

   1. Nunca usar `kernel=linear`, usar las clases específicas `LinearSVC` y `LinearSVR`
   
   2. `kernel=rbf` suele funcionar bien en la mayoría de los casos
   
   3. Para `SVC`, ajustar los valores de `C` y `gamma` por validación cruzada
   
   4. Para `SVR`, ajusta los valores de `C`, `gamma` y `epsilon` por validación cruzada
   
   5. Aumentar el valor de `cache_size` a 500Mb o incluso 1Gb si hay suficiente RAM disponible puede hacer que el entrenamiento vaya más rápido
   
Es habitual hacer la búsqueda en las siguientes rejillas: 
   
   * $\text{C}=2^{-5}, 2^{-3}, \dots, 2^{13}, 2^{15}$
   
   
   * $\text{gamma}=2^{-15}, 2^{-13}, \dots, 2^{1}, 2^{3}$
   
   
   * $\text{epsilon}=2^{-8}, 2^{-7}, \dots, 2^{-2}, 2^{-1}$

In [3]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

param_grid = {'C': 2.0**np.arange(-5, 16, 2),
              'gamma': 2.0**np.arange(-15, 4, 2)}

grid = GridSearchCV(estimator=SVC(), param_grid=param_grid, cv=5)

In [4]:
grid.fit(X_train, y_train)
grid.best_params_

{'C': 8.0, 'gamma': 3.0517578125e-05}

In [5]:
grid.score(X_test, y_test)

0.951048951048951

In [6]:
svc = SVC()
svc.fit(X_train, y_train)
svc.score(X_test, y_test)

0.9370629370629371

**Nota:** Es comveniente comprobar los resultados completos de la búsqueda para ver que los parámetros óptimos no están en los extremos de la rejilla que hemos creado

### SVM multiclase

Las SVM son clasificadores binarios. Para extenderlas a multiclase, se utiliza el esquema *one-vs-one* (OvO):

  * se construye un clasificador por cada par de clases 
  
  
  * las predicciones se combinan usando la clase mayoritaria
  
 <img src=../../img/OVO.jpg width=500>

### Ejercicios

* Ajustar una SVR al conjunto de datos `AmesHousing.csv`. Estandariza primero las variables para que estén en el intervalo $[-1, 1]$

* Busca ahora el valor óptimo de los parámetros `C`, `gamma` y `epsilon` usando validación cruzada y búsqueda en rejilla (`GridSearchCV`)